## Deploying the model as a service 

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#!pip install wandb

In [3]:
#!pip install bentoml

#### Notes:

- Ensure that you have the same version of sklearn as the version on which the pickle file was created.

- Check the sklearn version of the Google Colab platform
- Check the sklearn version here as below

Otherwise, update the sklearn version using

<code> pip install -U scikit-learn==1.0.2 </code>

Assuming, 1.0.2 is the version sklearn using which the pickle file was created. Change the version number accordingly after checking the version on Google Colab.

In [4]:
import sklearn

sklearn.__version__

'1.0.2'

In [5]:
import os
import wandb
import pandas as pd
import numpy as np

## Download the model from Weights and Biases

The model is stored under the following information

- **Project name :** 'mlops_usedcar'
- **Path to the model:** 'awesomestats/mlops_usedcar/Linear_Model_UsedCar:v2'

**Note**:

- Replace the key with your key of weights&biases (wandb)


In [6]:
import os
os.environ["WANDB_API_KEY"] = "dc797f600c763f09dc0ffc8637c8bcdf5bc1294b"

In [7]:
run = wandb.init(project='mlops_usedcar')

wandb: Currently logged in as: awesomestats. Use `wandb login --relogin` to force relogin


In [8]:
artifact = run.use_artifact('awesomestats/mlops_usedcar/Linear_Model_UsedCar:v0', type='model')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


In [9]:
f"the artifact is downloaded and stored at {artifact_dir}"

'the artifact is downloaded and stored at ./artifacts/Linear_Model_UsedCar:v0'

In [10]:
!ls -al  ./artifacts/Linear_Model_UsedCar:v0

total 24
drwxr-xr-x  3 manaranjan  staff    96 Dec 12 15:20 .
drwxr-xr-x  4 manaranjan  staff   128 Dec 12 15:20 ..
-rw-r--r--  1 manaranjan  staff  8655 Dec 12 15:20 cars.pkl


In [11]:
import joblib

In [12]:
joblib.__version__

'1.2.0'

### Load the Pickel file into Memory

In [13]:
from joblib import load

final_model = load('./artifacts/Linear_Model_UsedCar:v0/cars.pkl')

## Creating the BentoML Model Service

In [14]:
import bentoml
from bentoml.io import JSON

In [15]:
bento_model = bentoml.sklearn.save_model( "usedcar_prediction_model", 
                                         final_model,
                                         signatures={
                                            "predict": {"batchable": True, 
                                                        "batch_dim": 0},
                                        })

In [23]:
%%writefile used_car_service.py

import bentoml
import pandas as pd
import numpy as np
from bentoml.io import JSON

bento_model = bentoml.sklearn.get("usedcar_prediction_model:latest")

model_runner = bento_model.to_runner()

svc = bentoml.Service("usedcar_price_predictor", 
                      runners=[model_runner])


@svc.api(input=JSON(), output=JSON())
async def predict(input_doc: str):
        
    df = pd.DataFrame(input_doc, index = [0])
    
    num_features = ['KM_Driven', 'age', 'power_new',
                    'Seats','mileage_new', 'engine_new']    

    df[num_features] = df[num_features].apply(pd.to_numeric, 
                                              errors='coerce')
    
    print(df.info())
    
    print(df.head(1))
    
    predictions = await model_runner.predict.async_run(df)
    
    return {"ExpectedSalePrice": np.round(predictions[0], 2)}

Overwriting used_car_service.py


## Build Bentoml service

In [34]:
!bentoml build

Building BentoML service "usedcar_price_predictor:3kpqpwd2bk7adgab" from build context "/Users/manaranjan/Documents/Work/IIMB/edx/MLOps/modeldeploy".
Packing model "usedcar_prediction_model:5mzgmad2aokn5gab"
Locking PyPI package versions.

██████╗░███████╗███╗░░██╗████████╗░█████╗░███╗░░░███╗██╗░░░░░
██╔══██╗██╔════╝████╗░██║╚══██╔══╝██╔══██╗████╗░████║██║░░░░░
██████╦╝█████╗░░██╔██╗██║░░░██║░░░██║░░██║██╔████╔██║██║░░░░░
██╔══██╗██╔══╝░░██║╚████║░░░██║░░░██║░░██║██║╚██╔╝██║██║░░░░░
██████╦╝███████╗██║░╚███║░░░██║░░░╚█████╔╝██║░╚═╝░██║███████╗
╚═════╝░╚══════╝╚═╝░░╚══╝░░░╚═╝░░░░╚════╝░╚═╝░░░░░╚═╝╚══════╝

Successfully built Bento(tag="usedcar_price_predictor:3kpqpwd2bk7adgab").


In [35]:
!bentoml list

 Tag                     Size       Creation Time        Path                   
 usedcar_price_predict…  23.76 KiB  2022-12-12 16:20:58  ~/bentoml/bentos/used… 
 usedcar_price_predict…  24.58 KiB  2022-12-12 15:48:41  ~/bentoml/bentos/used… 


## Creating a Docker Image

In [28]:
!docker images

REPOSITORY                     TAG                     IMAGE ID       CREATED         SIZE
usedcarprice-mlops-19feb2022   20220219101716_AC1251   ad750b50d9bf   9 months ago    1.28GB
usedcarprice-mlops-18feb2022   20220218150200_27AECF   39b0fe031551   9 months ago    1.28GB
bentoml/model-server           0.13.1-py38             f932eddc0ef4   17 months ago   1.16GB


In [37]:
!bentoml containerize usedcar_price_predictor:latest    

Building docker image for usedcar_price_predictor:3kpqpwd2bk7adgab
[+] Building 0.0s (0/0)                                                         
[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                

[+] Building 2.2s (4/4)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         2.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
[+] Building 2.3s (12/15)                                                       
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docke

[+] Building 3.1s (12/15)                                                       
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         2.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [base-debian  1/10] FROM docker.io/library/python:3.8-slim@sha256:e7e  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 25.37kB                                       0.0s
 => CACHED [base-debian  2/10] RUN rm -f /etc/apt/apt.conf.d/docker-clean  0.0s
 => CACHED [base-debian  3/10] RUN --mount=type=cache,target=/var/lib/apt  0.0s
 => CACHED [base-debian  4/10] RUN grou

[+] Building 3.7s (12/15)                                                       
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         2.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [base-debian  1/10] FROM docker.io/library/python:3.8-slim@sha256:e7e  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 25.37kB                                       0.0s
 => CACHED [base-debian  2/10] RUN rm -f /etc/apt/apt.conf.d/docker-clean  0.0s
 => CACHED [base-debian  3/10] RUN --mount=type=cache,target=/var/lib/apt  0.0s
 => CACHED [base-debian  4/10] RUN grou

[+] Building 4.3s (12/15)                                                       
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         2.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [base-debian  1/10] FROM docker.io/library/python:3.8-slim@sha256:e7e  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 25.37kB                                       0.0s
 => CACHED [base-debian  2/10] RUN rm -f /etc/apt/apt.conf.d/docker-clean  0.0s
 => CACHED [base-debian  3/10] RUN --mount=type=cache,target=/var/lib/apt  0.0s
 => CACHED [base-debian  4/10] RUN grou

...

[+] Building 47.1s (15/16)                                                      
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         2.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [base-debian  1/10] FROM docker.io/library/python:3.8-slim@sha256:e7e  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 25.37kB                                       0.0s
 => CACHED [base-debian  2/10] RUN rm -f /etc/apt/apt.conf.d/docker-clean  0.0s
 => CACHED [base-debian  3/10] RUN --mount=type=cache,target=/var/lib/apt  0.0s
 => CACHED [base-debian  4/10] RUN grou

[+] Building 47.9s (15/16)                                                      
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         2.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [base-debian  1/10] FROM docker.io/library/python:3.8-slim@sha256:e7e  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 25.37kB                                       0.0s
 => CACHED [base-debian  2/10] RUN rm -f /etc/apt/apt.conf.d/docker-clean  0.0s
 => CACHED [base-debian  3/10] RUN --mount=type=cache,target=/var/lib/apt  0.0s
 => CACHED [base-debian  4/10] RUN grou

[+] Building 48.6s (15/16)                                                      
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         2.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [base-debian  1/10] FROM docker.io/library/python:3.8-slim@sha256:e7e  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 25.37kB                                       0.0s
 => CACHED [base-debian  2/10] RUN rm -f /etc/apt/apt.conf.d/docker-clean  0.0s
 => CACHED [base-debian  3/10] RUN --mount=type=cache,target=/var/lib/apt  0.0s
 => CACHED [base-debian  4/10] RUN grou

[+] Building 49.4s (15/16)                                                      
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         2.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [base-debian  1/10] FROM docker.io/library/python:3.8-slim@sha256:e7e  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 25.37kB                                       0.0s
 => CACHED [base-debian  2/10] RUN rm -f /etc/apt/apt.conf.d/docker-clean  0.0s
 => CACHED [base-debian  3/10] RUN --mount=type=cache,target=/var/lib/apt  0.0s
 => CACHED [base-debian  4/10] RUN grou

[+] Building 50.1s (15/16)                                                      
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.82kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim         2.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [base-debian  1/10] FROM docker.io/library/python:3.8-slim@sha256:e7e  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 25.37kB                                       0.0s
 => CACHED [base-debian  2/10] RUN rm -f /etc/apt/apt.conf.d/docker-clean  0.0s
 => CACHED [base-debian  3/10] RUN --mount=type=cache,target=/var/lib/apt  0.0s
 => CACHED [base-debian  4/10] RUN grou

In [40]:
!docker images

REPOSITORY                     TAG                     IMAGE ID       CREATED          SIZE
usedcar_price_predictor        3kpqpwd2bk7adgab        015835ee903c   42 seconds ago   792MB
usedcar_price_predictor        krqabjd2a2aurgab        e0c5083e89e6   24 minutes ago   731MB
usedcarprice-mlops-19feb2022   20220219101716_AC1251   ad750b50d9bf   9 months ago     1.28GB
usedcarprice-mlops-18feb2022   20220218150200_27AECF   39b0fe031551   9 months ago     1.28GB
bentoml/model-server           0.13.1-py38             f932eddc0ef4   17 months ago    1.16GB


## Running the Docker Image as a Service

In [44]:
!docker run -p 3000:3000 usedcar_price_predictor:3kpqpwd2bk7adgab

2022-12-12T10:55:27+0000 [INFO] [cli] Service loaded from Bento directory: bentoml.Service(tag="usedcar_price_predictor:3kpqpwd2bk7adgab", path="/home/bentoml/bento/")
2022-12-12T10:55:27+0000 [INFO] [cli] Environ for worker 0: set CPU thread count to 4
2022-12-12T10:55:27+0000 [INFO] [cli] Prometheus metrics for HTTP BentoServer from "/home/bentoml/bento" can be accessed at http://localhost:3000/metrics.
2022-12-12T10:55:28+0000 [INFO] [cli] Starting production HTTP BentoServer from "/home/bentoml/bento" listening on http://0.0.0.0:3000 (Press CTRL+C to quit)
2022-12-12T10:55:29+0000 [INFO] [api_server:1] Service loaded from Bento directory: bentoml.Service(tag="usedcar_price_predictor:3kpqpwd2bk7adgab", path="/home/bentoml/bento/")
2022-12-12T10:55:29+0000 [INFO] [api_server:3] Service loaded from Bento directory: bentoml.Service(tag="usedcar_price_predictor:3kpqpwd2bk7adgab", path="/home/bentoml/bento/")
2022-12-12T10:55:29+0000 [INFO] [api_server:2] Service loaded from Bento direct